In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()

# Normalization of data to standard distribution
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.7, random_state=1)

n_features = X.shape[1]
n_classes = Y.shape[1]

In [12]:
import random
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

mutation_rate = 0.3

class ANN(Sequential):    
    def __init__(self, child_weights=None):
        super().__init__()

        if child_weights is None:
            layer1 = Dense(4, input_shape=(4,), activation='relu')
            layer2 = Dense(16, activation='relu')
            layer3 = Dense(3, activation='softmax')
            self.add(layer1)
            self.add(layer2)
            self.add(layer3)
        else:
            self.add(
                Dense(
                    4,
                    input_shape=(4,),
                    activation='relu',
                    weights=[child_weights[0], np.ones(4)])
                )
            self.add(
                Dense(
                    16,
                    activation='relu',
                    weights = [child_weights[1], np.ones(16)]
                )
            )
            self.add(
                Dense(
                    3,
                    activation='softmax',
                    weights=[child_weights[2], np.zeros(3)])
            )

    def forward_propagation(self, X_train, Y_train):
        predict_label = self.predict(X_train)
        self.fitness = accuracy_score(np.argmax(Y_train, axis=1), np.argmax(predict_label, axis=1))
    
    def compile_train(self, epochs):
        self.compile(
                      optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy']
                      )
        self.fit(X_train, Y_train, epochs=epochs)

def crossover(nn1, nn2):
    
    nn1_weights = []
    nn2_weights = []
    child_weights = []

    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])

    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])

    for i in range(len(nn1_weights)):
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]

        child_weights.append(nn1_weights[i])

    mutation(child_weights)

    child = ANN(child_weights)
    return child

def mutation(child_weights):
    selection = random.randint(0, len(child_weights)-1)
    mut = random.uniform(0, 1)
    if mut <= mutation_rate:
        child_weights[selection] *= random.randint(2, 15)
    else:
        pass


networks = []
pool = []
generation = 0
population = 50
for i in range(population):
    networks.append(ANN())
max_fitness = 0
optimal_weights = []

epochs = 20
#This loop runs evolution
for i in range(epochs):
    generation += 1

    for ann in networks:
        ann.forward_propagation(X_train, Y_train)
        pool.append(ann)

    networks.clear()

    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()

    for i in range(len(pool)):
        if pool[i].fitness > max_fitness:
            max_fitness = pool[i].fitness

            print(f"Max fitness: {max_fitness}")
            optimal_weights = []
            for layer in pool[i].layers:
                optimal_weights.append(layer.get_weights()[0])

    #Only top 25% of population propagate genes
    for i in range(population//4):
        for j in range(2):
            temp = crossover(pool[i], random.choice(pool))
            networks.append(temp)

# Create a Genetic Neural Network with optimal initial weights
ann = ANN(optimal_weights)
ann.compile_train(epochs)
predict_label = ann.predict(X_test)
print('Test Accuracy: %.2f' % accuracy_score(np.argmax(Y_test, axis=1), np.argmax(predict_label, axis=1)))

2/2 [==============================] - 0s 3ms/step
Max fitness: 0.8444444444444444
2/2 [==============================] - 0s 4ms/step
Epoch 1/20
2/2 [==============================] - 1s 12ms/step - loss: 1.2603 - accuracy: 0.3333
Epoch 2/20
2/2 [==============================] - 0s 9ms/step - loss: 1.2007 - accuracy: 0.3333
Epoch 3/20
2/2 [==============================] - 0s 9ms/step - loss: 1.1435 - accuracy: 0.3333
Epoch 4/20
2/2 [==============================] - 0s 9ms/step - loss: 1.0916 - accuracy: 0.3333
Epoch 5/20
2/2 [==============================] - 0s 6ms/step - loss: 1.0397 - accuracy: 0.3333
Epoch 6/20
2/2 [==============================] - 0s 9ms/step - loss: 0.9955 - accuracy: 0.3556
Epoch 7/20
2/2 [==============================] - 0s 9ms/step - loss: 0.9512 - accuracy: 0.4000
Epoch 8/20
2/2 [==============================] - 0s 9ms/step - loss: 0.9124 - accuracy: 0.4222
Epoch 9/20
2/2 [==============================] - 0s 8ms/step - loss: 0.8727 - accuracy: 0.4667
E